# Version

* `v1`: 
    classification: tf_efficientnetv2_m_in21k_fold0_epoch17_colab_pseudo_labeling_v5.pt, tf_efficientnetv2_m_in21k_fold1_epoch0_colab_v33.pt, tf_efficientnetv2_m_in21k_fold2_epoch6_colab_pseudo_labeling_v7.pt, tf_efficientnetv2_m_in21k_fold3_epoch15_colab_pseudo_labeling_v8.pt, tf_efficientnetv2_m_in21k_fold4_epoch26_colab_pseudo_labeling_v9.pt, vit_deit_base_patch16_384_fold0_epoch10_colab_vit_v7.pt, vit_deit_base_patch16_384_fold1_epoch9_colab_vit_v9.pt, vit_deit_base_patch16_384_fold2_epoch14_colab_vit_v11.pt, vit_deit_base_patch16_384_fold3_epoch6_colab_vit_v12.pt, vit_deit_base_patch16_384_fold4_epoch13_colab_vit_v13.pt weights=[38,38,38,38,38,37,37,37,37,37] 3_tta
    
    2-class: tf_efficientnet_v2 + vit_deit_base_patch16_384_fold0_epoch8_colab_v6.pt + vit_deit_base_patch16_384_fold1_epoch8_colab_v7.pt + vit_deit_base_patch16_384_fold2_epoch9_colab_v8.pt weights=[59,59,59,59,59,58,58,58] 3_tta
    
    detection: yolov5x_fold0_nb_v14_best.pt + yolov5x_fold1_nb_v19_best.pt + yolov5x_fold2_nb_v20_best.pt + siimcovid19-effdet/effdet5_640_fold3_epoch20.bin + siimcovid19-effdet/effdet5_640_fold4_epoch24.bin nms_0.5 img_640 + cascade_rcnn_x101_32x4d_fpn_1x_fold0_epoch10_public_nb_v8.pth nms_iou_0.5 wbf iou=0.6 weight=[54,54,54,54,54,58,58,58,58,58,54] skipbox_thr=0.01 tta (1 - none_score) ** 0.5 img_640

### Note:
- df_2class and sub_df should be merged by id, not sub_df['none'] = df_2class['none'] because there might be a mix of ids

thanks to https://www.kaggle.com/xhlulu/siim-covid-19-convert-to-jpg-256px  
thanks to https://www.kaggle.com/awsaf49/vinbigdata-cxr-ad-yolov5-14-class-infer  
train_study: https://www.kaggle.com/h053473666/siim-covid19-efnb7-train-study  
train_image: https://www.kaggle.com/h053473666/siim-cov19-yolov5-train  
train_2class: https://www.kaggle.com/h053473666/siim-covid19-efnb7-train-fold0-5-2class  
  
version1:Original hyperparameters (yolov5)  
version4:New hyperparameters (yolov5)


In [1]:
!conda install '/kaggle/input/pydicom-conda-helper/libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/libgcc-ng-9.3.0-h2828fa1_19.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/gdcm-2.8.9-py37h500ead1_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/conda-4.10.1-py37h89c1867_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/certifi-2020.12.5-py37h89c1867_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/openssl-1.1.1k-h7f98852_0.tar.bz2' -c conda-forge -y


######################################################################## | 100% 
Preparing transaction: - done
Verifying transaction: | done
Executing transaction: - \ done

######################################################################## | 100% 
Preparing transaction: - done
Verifying transaction: | done
Executing transaction: - \ | done

######################################################################## | 100% 
Preparing transaction: - done
Verifying transaction: | done
Executing transaction: - done

######################################################################## | 100% 
Preparing transaction: - done
Verifying transaction: | done
Executing transaction: - \ | / - \ | / - \ | / - \ | / - \ done

######################################################################## | 100% 
Preparing transaction: - done
Verifying transaction: | done
Executing transaction: - done

#############################

In [2]:
!pip install --no-deps '../input/pycocotools/pycocotools-2.0-cp37-cp37m-linux_x86_64.whl' > /dev/null

In [3]:
package_paths = [
    '../input/weighted-boxes-fusion/Weighted-Boxes-Fusion-master',
    '../input/pytorchimagemodels/pytorch-image-models-master', #'../input/efficientnet-pytorch-07/efficientnet_pytorch-0.7.0'
    "../input/timmeffdetclsv2/timm-efficientdet-pytorch",
    "../input/omegaconf"
#     '../input/image-fmix/FMix-master'
]
import sys; 

for pth in package_paths:
    sys.path.append(pth)

In [4]:
import os

from PIL import Image
import pandas as pd
from tqdm.auto import tqdm
from ensemble_boxes import *
import matplotlib.pyplot as plt

In [5]:
df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
if df.shape[0] == 2477:
    fast_sub = True
    fast_df = pd.DataFrame(([['00086460a852_study', 'negative 1 0 0 1 1'], 
                         ['000c9c05fd14_study', 'negative 1 0 0 1 1'], 
                         ['65761e66de9f_image', 'none 1 0 0 1 1'], 
                         ['51759b5579bc_image', 'none 1 0 0 1 1']]), 
                       columns=['id', 'PredictionString'])
else:
    fast_sub = False


# .dcm to .png

In [6]:
import numpy as np
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

def read_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data

In [7]:
def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    
    return im

In [8]:
split = 'test'
save_dir = f'/kaggle/tmp/{split}/'

os.makedirs(save_dir, exist_ok=True)

save_dir = f'/kaggle/tmp/{split}/study/'
os.makedirs(save_dir, exist_ok=True)
if fast_sub:
    xray = read_xray('../input/siim-covid19-detection/train/00086460a852/9e8302230c91/65761e66de9f.dcm')
    im = resize(xray, size=640)  
    study = '00086460a852' + '_study.png'
    im.save(os.path.join(save_dir, study))
    xray = read_xray('../input/siim-covid19-detection/train/000c9c05fd14/e555410bd2cd/51759b5579bc.dcm')
    im = resize(xray, size=640)  
    study = '000c9c05fd14' + '_study.png'
    im.save(os.path.join(save_dir, study))
else:   
    for dirname, _, filenames in tqdm(os.walk(f'../input/siim-covid19-detection/{split}')):
        for file in filenames:
            # set keep_ratio=True to have original aspect ratio
            xray = read_xray(os.path.join(dirname, file))
            im = resize(xray, size=640)  
            study = dirname.split('/')[-2] + '_study.png'
            im.save(os.path.join(save_dir, study))


In [9]:
image_id = []
dim0 = []
dim1 = []
splits = []
save_dir = f'/kaggle/tmp/{split}/image/'
os.makedirs(save_dir, exist_ok=True)
if fast_sub:
    xray = read_xray('../input/siim-covid19-detection/train/00086460a852/9e8302230c91/65761e66de9f.dcm')
    im = resize(xray, size=640)  
    im.save(os.path.join(save_dir,'65761e66de9f_image.png'))
    image_id.append('65761e66de9f.dcm'.replace('.dcm', ''))
    dim0.append(xray.shape[0])
    dim1.append(xray.shape[1])
    splits.append(split)
    xray = read_xray('../input/siim-covid19-detection/train/000c9c05fd14/e555410bd2cd/51759b5579bc.dcm')
    im = resize(xray, size=640)  
    im.save(os.path.join(save_dir, '51759b5579bc_image.png'))
    image_id.append('51759b5579bc.dcm'.replace('.dcm', ''))
    dim0.append(xray.shape[0])
    dim1.append(xray.shape[1])
    splits.append(split)
else:
    for dirname, _, filenames in tqdm(os.walk(f'../input/siim-covid19-detection/{split}')):
        for file in filenames:
            # set keep_ratio=True to have original aspect ratio
            xray = read_xray(os.path.join(dirname, file))
            im = resize(xray, size=640)  
            im.save(os.path.join(save_dir, file.replace('.dcm', '_image.png')))
            image_id.append(file.replace('.dcm', ''))
            dim0.append(xray.shape[0])
            dim1.append(xray.shape[1])
            splits.append(split)
meta = pd.DataFrame.from_dict({'image_id': image_id, 'dim0': dim0, 'dim1': dim1, 'split': splits})

# study predict

In [10]:
from glob import glob
from sklearn.model_selection import GroupKFold, StratifiedKFold
import cv2
from skimage import io
import torch
from torch import nn
import os
from datetime import datetime
import time
import random
import cv2
import torchvision
from torchvision import transforms
import pandas as pd
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from  torch.cuda.amp import autocast, GradScaler

import sklearn
import warnings
import joblib
from sklearn.metrics import roc_auc_score, log_loss
from sklearn import metrics
import warnings
import cv2
import pydicom
import timm #from efficientnet_pytorch import EfficientNet
from scipy.ndimage.interpolation import zoom
from sklearn.metrics import log_loss

In [11]:
CFG = {
    'fold_num': 5,
    'seed': 2011,
    'model_arch': 'tf_efficientnetv2_m_in21k',
    'vit_model_arch': 'vit_deit_base_patch16_384',
    'img_size': 640,
    'epochs': 10,
    'train_bs': 32,
    'valid_bs': 32,
    'lr': 1e-4,
    'num_workers': 4,
    'accum_iter': 1, # suppoprt to do batch accumulation for backprop with effectively larger batch size
    'verbose_step': 1,
    'device': 'cuda:0',
    'tta': 3,
    'model_paths': ['../input/siimcovid19-pseudo-labeling-classification/tf_efficientnetv2_m_in21k_fold0_epoch17_colab_pseudo_labeling_v5.pt',
                   '../input/siim-covid19-efficientnet/tf_efficientnetv2_m_in21k_fold1_epoch0_colab_v33.pt',
                   '../input/siimcovid19-pseudo-labeling-classification/tf_efficientnetv2_m_in21k_fold2_epoch6_colab_pseudo_labeling_v7.pt',
                   '../input/siimcovid19-pseudo-labeling-classification/tf_efficientnetv2_m_in21k_fold3_epoch15_colab_pseudo_labeling_v8.pt',
                   '../input/siimcovid19-pseudo-labeling-classification/tf_efficientnetv2_m_in21k_fold4_epoch26_colab_pseudo_labeling_v9.pt',
                   '../input/siimcovid19-vision-transformer-model/vit_deit_base_patch16_384_fold0_epoch10_colab_vit_v7.pt',
                   '../input/siimcovid19-vision-transformer-model/vit_deit_base_patch16_384_fold1_epoch9_colab_vit_v9.pt',
                   '../input/siimcovid19-vision-transformer-model/vit_deit_base_patch16_384_fold2_epoch14_colab_vit_v11.pt',
                   '../input/siimcovid19-vision-transformer-model/vit_deit_base_patch16_384_fold3_epoch6_colab_vit_v12.pt',
                   '../input/siimcovid19-vision-transformer-model/vit_deit_base_patch16_384_fold4_epoch13_colab_vit_v13.pt'],
    'num_classes': 4,
    'weights': [38, 38, 38, 38, 38, 37, 37, 37, 37, 37]
}

In [12]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
def get_img(path):
    im_bgr = cv2.imread(path)
    im_rgb = im_bgr[:, :, ::-1]
    #print(im_rgb)
    return im_rgb

# img = get_img('../input/siim-covid19-resized-to-512px-png/test/0026720152f5.png')
# plt.imshow(img)
# plt.show()

## Dataset

In [13]:
class ChestXRayDataset(Dataset):
    def __init__(
        self, df, data_root, transforms=None, output_label=True
    ):
        
        super().__init__()
        self.df = df.reset_index(drop=True).copy()
        self.transforms = transforms
        self.data_root = data_root
        self.output_label = output_label
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index: int):
        
        # get labels
        if self.output_label:
            target = self.df.iloc[index]['target']
        
        study_id = self.df.loc[index]['image_id']
        image_path = "{}/{}.png".format(self.data_root, study_id)
        if (image_path.find('.png') == -1):
            image_path = image_path + '.png'
        
        img = get_img(image_path)
        
        if self.transforms:
            img = self.transforms(image=img)['image']
            
        # do label smoothing
        if self.output_label == True:
            return img, target
        else:
            return img

## Define Train\Validation Image Augmentations

In [14]:
from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize
)

from albumentations.pytorch import ToTensorV2

def get_train_transforms():
    return Compose([
            RandomResizedCrop(CFG['img_size'], CFG['img_size']),
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            #VerticalFlip(p=0.5),
            ShiftScaleRotate(p=0.5),
            HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            #CoarseDropout(p=0.5),
            #Cutout(p=0.5),
            ToTensorV2(p=1.0),
        ], p=1.)
  
        
def get_valid_transforms():
    return Compose([
            CenterCrop(CFG['img_size'], CFG['img_size'], p=1.),
            Resize(CFG['img_size'], CFG['img_size']),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)

def get_inference_transforms():
    return Compose([
            RandomResizedCrop(CFG['img_size'], CFG['img_size'], scale=(0.8, 1.0)),
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)

# def get_inference_transforms():
#     return Compose([
#             Resize(CFG['img_size'], CFG['img_size']),
#             Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
#             ToTensorV2(p=1.0),
#         ], p=1.)

## Model

In [15]:
class ChestXrayImgClassifier(nn.Module):
    def __init__(self, model_arch, n_class, pretrained=False):
        super().__init__()
        # tf_efficientnet_b6_ns
        self.model = timm.create_model(model_arch, pretrained=pretrained, drop_rate=0.5, drop_path_rate=0.2)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(n_features, n_class, bias=True)
        )
        
    def forward(self, x):
        x = self.model(x)
        return x

In [16]:
class ChestXrayImgClassifierVIT(nn.Module):
    def __init__(self, model_arch, n_class, pretrained=False):
        super().__init__()
        # vit_base_patch16
        self.model = timm.create_model(model_arch, pretrained=pretrained, img_size=640)
        n_features = self.model.head.in_features
        self.model.head = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(n_features, n_class, bias=True)
        )

    def forward(self, x):
        x = self.model(x)
        return x

## Main Loop

In [17]:
import numpy as np 
import pandas as pd

if fast_sub:
    df = fast_df.copy()
else:
    df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
id_laststr_list  = []
for i in range(df.shape[0]):
    id_laststr_list.append(df.loc[i,'id'][-1])
df['id_last_str'] = id_laststr_list

study_len = df[df['id_last_str'] == 'y'].shape[0]

In [18]:
df

,id,PredictionString,id_last_str
0,00086460a852_study,negative 1 0 0 1 1,y
1,000c9c05fd14_study,negative 1 0 0 1 1,y
2,65761e66de9f_image,none 1 0 0 1 1,e
3,51759b5579bc_image,none 1 0 0 1 1,e


In [19]:
study_lst = df[df['id_last_str'] == 'y']['id'].values
study_lst

array(['00086460a852_study', '000c9c05fd14_study'], dtype=object)

In [20]:
def inference_one_epoch(model, data_loader, device):
    model.eval()

    image_preds_all = []
    
    pbar = tqdm(enumerate(data_loader), total=len(data_loader))
    for step, (imgs) in pbar:
        imgs = imgs.to(device).float()
        
        image_preds = model(imgs)   #output = model(input)
        image_preds_all += [torch.softmax(image_preds, 1).detach().cpu().numpy()]
        
    
    image_preds_all = np.concatenate(image_preds_all, axis=0)
    return image_preds_all

In [21]:
fold = 0
test_dataset_dir = f'/kaggle/tmp/{split}/study'
 # for training only, need nightly build pytorch

seed_everything(CFG['seed'])

print('Inference fold {} started'.format(fold))

test = pd.DataFrame()
test['image_id'] = study_lst
test_ds = ChestXRayDataset(test, test_dataset_dir, transforms=get_inference_transforms(), output_label=False)

tst_loader = torch.utils.data.DataLoader(
    test_ds, 
    batch_size=CFG['valid_bs'],
    num_workers=CFG['num_workers'],
    shuffle=False,
    pin_memory=False,
)

device = torch.device(CFG['device'])
model = ChestXrayImgClassifier(CFG['model_arch'], CFG['num_classes']).to(device)

tst_preds = []

for i, model_path in enumerate(CFG['model_paths']):
    if i > 4:
        break
    model.load_state_dict(torch.load(model_path))

    with torch.no_grad():
        for _ in range(CFG['tta']):
            infer_one = inference_one_epoch(model, tst_loader, device)
    #         print(infer_one)
            tst_preds += [CFG['weights'][i]/sum(CFG['weights'])/CFG['tta']*infer_one]

# print('tst_preds')
# print(tst_preds[:10])
# print(len(tst_preds))
# tst_preds = np.sum(tst_preds, axis=0)
# print(tst_preds[:10])
# print(len(tst_preds))

del model
torch.cuda.empty_cache()

Inference fold 0 started


100%|██████████| 1/1 [00:00<00:00,  4.34it/s]


In [22]:
model_vit = ChestXrayImgClassifierVIT(CFG['vit_model_arch'], CFG['num_classes']).to(device)
VIT_model_paths = CFG['model_paths'][5:]

for i, model_path in enumerate(VIT_model_paths):
    model_vit.load_state_dict(torch.load(model_path))

    with torch.no_grad():
        for _ in range(CFG['tta']):
            infer_one = inference_one_epoch(model_vit, tst_loader, device)
    #         print(infer_one)
            tst_preds += [CFG['weights'][i+5]/sum(CFG['weights'])/CFG['tta']*infer_one]

tst_preds = np.sum(tst_preds, axis=0)
del model_vit
torch.cuda.empty_cache() 

100%|██████████| 1/1 [00:00<00:00,  2.46it/s]


In [23]:
print(len(tst_preds))

2


In [24]:
tst_preds[:10]

array([[0.1119146 , 0.22203572, 0.42803884, 0.2380108 ],
       [0.04202302, 0.05283254, 0.03080076, 0.87434375]], dtype=float32)

In [25]:
debug = False
class_labels = ['0', '1', '2', '3']

test.loc[:99 if debug else test.shape[0], class_labels] = tst_preds
test.head()

,image_id,0,1,2,3
0,00086460a852_study,0.111915,0.222036,0.428039,0.238011
1,000c9c05fd14_study,0.042023,0.052833,0.030801,0.874344


# study string

In [26]:
name2label = { 
    'negative': 0,
    'indeterminate': 1,
    'typical': 2,
    'atypical': 3}
label2name  = {v:k for k, v in name2label.items()}

In [27]:
def get_PredictionString(row, thr=0):
    string = ''
    for idx in range(4):
        conf =  row[str(idx)]
        if conf>thr:
            string+=f'{label2name[idx]} {conf:0.3f} 0 0 1 1 '
    string = string.strip()
    return string

In [28]:
test['PredictionString'] = test.apply(get_PredictionString, axis=1)
test = test.drop(class_labels, axis=1)
test.rename(columns={'image_id':'id'}, inplace=True)
test.head()

,id,PredictionString
0,00086460a852_study,negative 0.112 0 0 1 1 indeterminate 0.222 0 0...
1,000c9c05fd14_study,negative 0.042 0 0 1 1 indeterminate 0.053 0 0...


In [29]:
df_study = test
df_study

,id,PredictionString
0,00086460a852_study,negative 0.112 0 0 1 1 indeterminate 0.222 0 0...
1,000c9c05fd14_study,negative 0.042 0 0 1 1 indeterminate 0.053 0 0...


# 2 class

In [30]:
package_paths = [
    '../input/pytorchimagemodels/pytorch-image-models-master', #'../input/efficientnet-pytorch-07/efficientnet_pytorch-0.7.0'
#     '../input/image-fmix/FMix-master'
]
import sys; 

for pth in package_paths:
    sys.path.append(pth)
    
# from fmix import sample_mask, make_low_freq_image, binarise_mask

In [31]:
from glob import glob
from sklearn.model_selection import GroupKFold, StratifiedKFold
import cv2
from skimage import io
import torch
from torch import nn
import os
from datetime import datetime
import time
import random
import cv2
import torchvision
from torchvision import transforms
import pandas as pd
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from  torch.cuda.amp import autocast, GradScaler

import sklearn
import warnings
import joblib
from sklearn.metrics import roc_auc_score, log_loss
from sklearn import metrics
import warnings
import cv2
import pydicom
import timm #from efficientnet_pytorch import EfficientNet
from scipy.ndimage.interpolation import zoom
from sklearn.metrics import log_loss

In [32]:
CFG = {
    'fold_num': 5,
    'seed': 2021,
    'model_arch': 'tf_efficientnetv2_m_in21k',
    'vit_model_arch': 'vit_deit_base_patch16_384',
    'img_size': 640,
    'epochs': 10,
    'train_bs': 32,
    'valid_bs': 32,
    'lr': 1e-4,
    'num_workers': 4,
    'accum_iter': 1, # suppoprt to do batch accumulation for backprop with effectively larger batch size
    'verbose_step': 1,
    'device': 'cuda:0',
    'tta': 3,
    'model_paths': ['../input/siimcovid192classes-classifier/tf_efficientnetv2_m_in21k_fold0_epoch1_colab_v1.pt',
                   '../input/siimcovid192classes-classifier/tf_efficientnetv2_m_in21k_fold1_epoch20_colab_v2.pt',
                   '../input/siimcovid192classes-classifier/tf_efficientnetv2_m_in21k_fold2_epoch14_colab_v3.pt',
                   '../input/siimcovid192classes-classifier/tf_efficientnetv2_m_in21k_fold3_epoch13_colab_v4.pt',
                   '../input/siimcovid192classes-classifier/tf_efficientnetv2_m_in21k_fold4_epoch5_colab_v5.pt',
                   '../input/siimcovid192classes-classifier/vit_deit_base_patch16_384_fold0_epoch8_colab_v6.pt',
                   '../input/siimcovid192classes-classifier/vit_deit_base_patch16_384_fold1_epoch8_colab_v7.pt',
                   '../input/siimcovid192classes-classifier/vit_deit_base_patch16_384_fold2_epoch9_colab_v8.pt'],
    'num_classes': 2,
    'weights': [59, 59, 59, 59, 59, 58, 58, 58]
}

In [33]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
def get_img(path):
    im_bgr = cv2.imread(path)
    im_rgb = im_bgr[:, :, ::-1]
    #print(im_rgb)
    return im_rgb

In [34]:
from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize
)

from albumentations.pytorch import ToTensorV2

def get_train_transforms():
    return Compose([
            RandomResizedCrop(CFG['img_size'], CFG['img_size']),
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            #VerticalFlip(p=0.5),
            ShiftScaleRotate(p=0.5),
            HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            #CoarseDropout(p=0.5),
            #Cutout(p=0.5),
            ToTensorV2(p=1.0),
        ], p=1.)
  
        
def get_valid_transforms():
    return Compose([
            CenterCrop(CFG['img_size'], CFG['img_size'], p=1.),
            Resize(CFG['img_size'], CFG['img_size']),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)

def get_inference_transforms():
    return Compose([
            RandomResizedCrop(CFG['img_size'], CFG['img_size'], scale=(0.8, 1.0)),
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)

# def get_inference_transforms():
#     return Compose([
#             Resize(CFG['img_size'], CFG['img_size']),
#             Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
#             ToTensorV2(p=1.0),
#         ], p=1.)

In [35]:
class ChestXrayImgClassifier(nn.Module):
    def __init__(self, model_arch, n_class, pretrained=False):
        super().__init__()
        # tf_efficientnet_b6_ns
        self.model = timm.create_model(model_arch, pretrained=pretrained, drop_rate=0.3, drop_path_rate=0.2)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(n_features, n_class, bias=True)
        )
        
    def forward(self, x):
        x = self.model(x)
        return x

In [36]:
def inference_one_epoch(model, data_loader, device):
    model.eval()

    image_preds_all = []
    
    pbar = tqdm(enumerate(data_loader), total=len(data_loader))
    for step, (imgs) in pbar:
        imgs = imgs.to(device).float()
        
        image_preds = model(imgs)   #output = model(input)
        image_preds_all += [torch.softmax(image_preds, 1).detach().cpu().numpy()]
        
    
    image_preds_all = np.concatenate(image_preds_all, axis=0)
    return image_preds_all

In [37]:
class ChestXRayDataset(Dataset):
    def __init__(
        self, df, data_root, transforms=None, output_label=True
    ):
        
        super().__init__()
        self.df = df.reset_index(drop=True).copy()
        self.transforms = transforms
        self.data_root = data_root
        self.output_label = output_label
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index: int):
        
        # get labels
        if self.output_label:
            target = self.df.iloc[index]['target']
        
        image_id = self.df.loc[index]['id']
        image_path = "{}/{}".format(self.data_root, image_id)
        if (image_path.find('.png') == -1):
            image_path = image_path + '.png'
        img = get_img(image_path)
        
        if self.transforms:
            img = self.transforms(image=img)['image']
            
        # do label smoothing
        if self.output_label == True:
            return img, target
        else:
            return img

In [38]:
if fast_sub:
    sub_df = fast_df.copy()
else:
    sub_df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
sub_df = sub_df[study_len:]
test_paths = f'/kaggle/tmp/{split}/image/' + sub_df['id'] +'.png'
sub_df['none'] = 0
sub_df

,id,PredictionString,none
2,65761e66de9f_image,none 1 0 0 1 1,0
3,51759b5579bc_image,none 1 0 0 1 1,0


In [39]:
fold = 0
test_dataset_dir = f'/kaggle/tmp/{split}/image'
 # for training only, need nightly build pytorch

seed_everything(CFG['seed'])

print('Inference fold {} started'.format(fold))

test = pd.DataFrame()
test_ds = ChestXRayDataset(sub_df, test_dataset_dir, transforms=get_inference_transforms(), output_label=False)

tst_loader = torch.utils.data.DataLoader(
    test_ds, 
    batch_size=CFG['valid_bs'],
    num_workers=CFG['num_workers'],
    shuffle=False,
    pin_memory=False,
)

device = torch.device(CFG['device'])
model = ChestXrayImgClassifier(CFG['model_arch'], CFG['num_classes']).to(device)

tst_preds = []

for i, model_path in enumerate(CFG['model_paths']): 
    if i > 4:
        break
    model.load_state_dict(torch.load(model_path))

    with torch.no_grad():
        for _ in range(CFG['tta']):
            infer_one = inference_one_epoch(model, tst_loader, device)
    #         print(infer_one)
            tst_preds += [CFG['weights'][i]/sum(CFG['weights'])/CFG['tta']*infer_one]

# tst_preds = np.sum(tst_preds, axis=0)

del model
torch.cuda.empty_cache()

Inference fold 0 started


100%|██████████| 1/1 [00:00<00:00,  4.52it/s]


In [40]:
model_vit = ChestXrayImgClassifierVIT(CFG['vit_model_arch'], CFG['num_classes']).to(device)
VIT_model_paths = CFG['model_paths'][5:]

for i, model_path in enumerate(VIT_model_paths):
    model_vit.load_state_dict(torch.load(model_path))

    with torch.no_grad():
        for _ in range(CFG['tta']):
            infer_one = inference_one_epoch(model_vit, tst_loader, device)
    #         print(infer_one)
            tst_preds += [CFG['weights'][i+5]/sum(CFG['weights'])/CFG['tta']*infer_one]

tst_preds = np.sum(tst_preds, axis=0)
del model_vit
torch.cuda.empty_cache() 

100%|██████████| 1/1 [00:00<00:00,  3.16it/s]


In [41]:
tst_preds

array([[0.87053233, 0.12946773],
       [0.2464764 , 0.7535236 ]], dtype=float32)

In [42]:
sub_df

,id,PredictionString,none
2,65761e66de9f_image,none 1 0 0 1 1,0
3,51759b5579bc_image,none 1 0 0 1 1,0


In [43]:
debug = False
class_labels = ['none']

sub_df[class_labels] = tst_preds[:, 1]
sub_df.head()

,id,PredictionString,none
2,65761e66de9f_image,none 1 0 0 1 1,0.129468
3,51759b5579bc_image,none 1 0 0 1 1,0.753524


In [44]:
df_2class = sub_df.reset_index(drop=True)
df_2class.head()

,id,PredictionString,none
0,65761e66de9f_image,none 1 0 0 1 1,0.129468
1,51759b5579bc_image,none 1 0 0 1 1,0.753524


# Effdet predict

In [45]:
%load_ext autoreload
%autoreload 2

In [46]:
%cd /kaggle/working/

/kaggle/working


In [47]:
from ensemble_boxes import *
import torch
import os
from datetime import datetime
import time
import random
import cv2
import pandas as pd
import numpy as np
import albumentations as A
import matplotlib.pyplot as plt
from albumentations.pytorch.transforms import ToTensorV2
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from glob import glob
import gc

SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

In [48]:
meta_copy = meta.copy()
meta_copy

,image_id,dim0,dim1,split
0,65761e66de9f,2330,2783,test
1,51759b5579bc,3093,2850,test


In [49]:
meta = meta_copy.copy()
meta = meta[meta['split'] == 'test']
if fast_sub:
    test_df = fast_df.copy()
else:
    test_df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
test_df = df[study_len:].reset_index(drop=True) 
meta['image_id'] = meta['image_id'] + '_image'
meta.columns = ['id', 'dim0', 'dim1', 'split']
# print(test_df)
test_df = pd.merge(test_df, meta, on = 'id', how = 'left')

In [50]:
test_df

,id,PredictionString,id_last_str,dim0,dim1,split
0,65761e66de9f_image,none 1 0 0 1 1,e,2330,2783,test
1,51759b5579bc_image,none 1 0 0 1 1,e,3093,2850,test


In [51]:
def get_valid_transforms_effdet():
    return A.Compose(
        [
            A.Resize(height=640, width=640, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)

In [52]:
DATA_ROOT_PATH = f'/kaggle/tmp/{split}/image/'

class DatasetRetriever(Dataset):
    def __init__(self, image_ids, transforms=None):
        super().__init__()
        self.image_ids = image_ids
        self.transforms = transforms

    def __getitem__(self, index: int):
        image_id = self.image_ids[index]
        image = cv2.imread(f'{DATA_ROOT_PATH}/{image_id}.png', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        if self.transforms:
            sample = {'image': image}
            sample = self.transforms(**sample)
            image = sample['image']
        return image, image_id

    def __len__(self) -> int:
        return self.image_ids.shape[0]

In [53]:
validation_dataset = DatasetRetriever(
    image_ids=meta["id"].values,
    transforms=get_valid_transforms_effdet()
)

In [54]:
validation_dataset.image_ids

array(['65761e66de9f_image', '51759b5579bc_image'], dtype=object)

In [55]:
!ls /kaggle/tmp/test/image

51759b5579bc_image.png	65761e66de9f_image.png


In [56]:
import warnings
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

SKIP_BOX_THR = 0.001
IMAGE_ID_TO_SIZE = {}
for i in range(len(meta)):
    IMAGE_ID_TO_SIZE[meta.at[i, "id"]] = {"width": meta.at[i, "dim1"], "height": meta.at[i, "dim0"]}
    
def get_boxes_classes_from_preds(preds, score_threshold=SKIP_BOX_THR):
    det = preds
    predictions = []
    for i in range(len(det)):
        boxes = det[i].detach().cpu().numpy()[:,:4]    
        scores = det[i].detach().cpu().numpy()[:,4]
        indexes = np.where(scores > score_threshold)[0]
        boxes = boxes[indexes]
        boxes[:, 2] = boxes[:, 2] + boxes[:, 0]
        boxes[:, 3] = boxes[:, 3] + boxes[:, 1]
        predictions.append({
            'boxes': boxes[indexes],
            'scores': scores[indexes],
        })
    return [predictions]


def run_wbf(predictions, image_index, image_size=512, iou_thr=0.44, skip_box_thr=SKIP_BOX_THR, weights=None):
    boxes = [(prediction[image_index]['boxes']/(image_size-1)).tolist()  for prediction in predictions]
    scores = [prediction[image_index]['scores'].tolist()  for prediction in predictions]
    labels = [np.ones(prediction[image_index]['scores'].shape[0]).tolist() for prediction in predictions]
    boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    boxes = boxes*(image_size-1)
    return boxes, scores, labels

def run_nms(predictions, image_index, image_size=640, iou_thr=0.5, weights=None):
    boxes = [(prediction[image_index]['boxes']/(image_size-1)).tolist()  for prediction in predictions]
    scores = [prediction[image_index]['scores'].tolist()  for prediction in predictions]
    labels = [np.ones(prediction[image_index]['scores'].shape[0]).tolist() for prediction in predictions]
    boxes, scores, labels = nms(boxes, scores, labels, weights=None, iou_thr=iou_thr)
    boxes = boxes*(image_size-1)
    return boxes, scores, labels

def box_label_conf_to_pred_str(boxes, labels, confs):
    pred_str = []
    for i in range(len(boxes)):
        x1, y1, x2, y2 = boxes[i]
        pred_str.append(f"{labels[i]} {confs[i]} {x1} {y1} {x2} {y2}")
    pred_str = " ".join(pred_str)
    return pred_str

def box_to_original_size(boxes, image_id, image_id_to_size, imsize=512):
    boxes = boxes.copy()
    for i, box in enumerate(boxes):
        w = image_id_to_size[image_id]["width"]
        h = image_id_to_size[image_id]["height"]
        box[[0,2]] = box[[0,2]] / imsize * w
        box[[1,3]] = box[[1,3]] / imsize * h
        boxes[i] = box
    return boxes    

class Fitter:
    def __init__(self, model, device, config):
        self.config = config

        self.base_dir = f'./{config.folder}'
        if not os.path.exists(self.base_dir):
            os.makedirs(self.base_dir)
        
        self.log_path = f'{self.base_dir}/log.txt'

        self.model = model
        self.device = device

        self.log(f'Fitter prepared. Device is {self.device}')

    def infer(self, val_loader, iou_threshold, visualize=False):
        print("iou threshold:", iou_threshold)
        self.log(f"iou threshold: {iou_threshold}")
        self.model.eval()
        t = time.time()
        results = []
        for step, (images, image_ids) in tqdm(enumerate(val_loader)):            
            with torch.no_grad():
                images = torch.stack(images)
                batch_size = images.shape[0]
                images = images.to(self.device).float()

                pred = self.model(images, torch.tensor([1]*images.shape[0]).float().cuda())
                
                predictions = get_boxes_classes_from_preds(pred)
                for i, image in enumerate(images):
#                     boxes, scores, labels = run_wbf(predictions, image_index=i, iou_thr=iou_threshold)
                    boxes, scores, labels = run_nms(predictions, image_index=i, iou_thr=iou_threshold)
                    boxes = boxes.astype(np.int32).clip(min=0, max=640)
                    image_id = image_ids[i]
                    text_labels = ["opacity"] * len(boxes)

                    boxes_ori_size = box_to_original_size(boxes, image_id, IMAGE_ID_TO_SIZE, imsize=640)
                    result = {
                        'id': image_id,
                        'PredictionString': box_label_conf_to_pred_str(boxes_ori_size, text_labels, scores)
                    }
                    results.append(result)
                    

                    #visualize
                    if visualize:
                      if i==0 and np.random.uniform() < 0.02:
                          sample = images[i].permute(1,2,0).cpu().numpy()
                          sample = np.ascontiguousarray(sample)
                    
                          fig, ax = plt.subplots(1, 1, figsize=(16, 8))

                          for score, box in zip(scores, boxes):
                              sample[int(box[1]):int(box[3]), int(box[0]):int(box[2]), 1] += score

                          for score, box in zip(scores, boxes):
                              sample = cv2.rectangle(sample, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (1, 0, 0, min(score*3,1)), 2)
                              
                          ax.set_axis_off()
                          ax.imshow(sample)
                          plt.show()
                          print(result)
        
        result_df = pd.DataFrame(results, columns=['id', 'PredictionString'])
        os.makedirs('/kaggle/working/effdet_results', exist_ok=True)
        result_file = f'/kaggle/working/effdet_results/result.csv'
        result_df.to_csv(result_file, index=False)
        return result_df

    def log(self, message):
        if self.config.verbose:
            print(message)
        with open(self.log_path, 'a+') as logger:
            logger.write(f'{message}\n')

In [57]:
class TrainGlobalConfig:
    num_workers = 2
    batch_size = 4
    folder = 'effdet5-cutmix-augmix'
    verbose = True
    verbose_step = 10

In [58]:
def collate_fn(batch):
    return tuple(zip(*batch))

def run_inference():
    device = torch.device('cuda:0')
    net.to(device)

    val_loader = torch.utils.data.DataLoader(
        validation_dataset, 
        batch_size=TrainGlobalConfig.batch_size,
        num_workers=TrainGlobalConfig.num_workers,
        shuffle=False,
        sampler=SequentialSampler(validation_dataset),
        pin_memory=False,
        collate_fn=collate_fn,
    )

    fitter = Fitter(model=net, device=device, config=TrainGlobalConfig)
    result_df = fitter.infer(val_loader, iou_threshold=0.5, visualize=True)
    return result_df

In [59]:
from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain, DetBenchEval
from effdet.efficientdet import HeadNet

def load_net(checkpoint_path):
    config = get_efficientdet_config('tf_efficientdet_d5')
    net = EfficientDet(config, pretrained_backbone=False)

    config.num_classes = 1
    config.image_size=640
    net.class_net = HeadNet(config, num_outputs=config.num_classes, norm_kwargs=dict(eps=.001, momentum=.01))

    checkpoint = torch.load(checkpoint_path)
    net.load_state_dict(checkpoint['model_state_dict'])

    del checkpoint
    gc.collect()

    net = DetBenchEval(net, config)
    net.eval();
    return net.cuda()


In [60]:
# net = load_net('/kaggle/input/siimcovid19-effdet/effdet5_640_fold0_epoch20.bin')
# effdet_fold0 = run_inference()
# effdet_fold0

In [61]:
# net = load_net('/kaggle/input/siimcovid19-effdet/effdet5_640_fold1_epoch21.bin')
# effdet_fold1 = run_inference()
# effdet_fold1

In [62]:
# net = load_net('/kaggle/input/siimcovid19-effdet/effdet5_640_fold2_epoch24.bin')
# effdet_fold2 = run_inference()
# effdet_fold2

In [63]:
net = load_net('/kaggle/input/siimcovid19-effdet/effdet5_640_fold3_epoch20.bin')
effdet_fold3 = run_inference()
effdet_fold3

Fitter prepared. Device is cuda:0
iou threshold: 0.5
iou threshold: 0.5


0it [00:00, ?it/s]

Warning. Fixed 3 boxes coordinates < 0
Warning. Fixed 5 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]


1it [00:04,  4.24s/it]

Warning. Fixed 1 boxes coordinates < 0
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]


,id,PredictionString
0,65761e66de9f_image,opacity 0.38977867364883423 1817 888 2430 1521...
1,51759b5579bc_image,opacity 0.4435000419616699 169 1908 1055 2479 ...


In [64]:
net = load_net('/kaggle/input/siimcovid19-effdet/effdet5_640_fold4_epoch24.bin')
effdet_fold4 = run_inference()
effdet_fold4

Fitter prepared. Device is cuda:0
iou threshold: 0.5
iou threshold: 0.5


1it [00:00,  3.98it/s]

Warning. Fixed 1 boxes coordinates < 0
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates < 0
Warning. Fixed 4 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]


,id,PredictionString
0,65761e66de9f_image,opacity 0.40472644567489624 1839 811 2417 1518...
1,51759b5579bc_image,opacity 0.42967167496681213 173 1899 1059 2498...


# yolov5 predict

In [65]:
import numpy as np, pandas as pd
from glob import glob
import shutil, os
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm
import seaborn as sns
import torch

In [66]:
meta

,id,dim0,dim1,split
0,65761e66de9f_image,2330,2783,test
1,51759b5579bc_image,3093,2850,test


In [67]:
meta = meta[meta['split'] == 'test']
if fast_sub:
    test_df = fast_df.copy()
else:
    test_df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
test_df = df[study_len:].reset_index(drop=True) 
# meta['image_id'] = meta['image_id'] + '_image'
# meta.columns = ['id', 'dim0', 'dim1', 'split']
test_df = pd.merge(test_df, meta, on = 'id', how = 'left')
test_df.head()

,id,PredictionString,id_last_str,dim0,dim1,split
0,65761e66de9f_image,none 1 0 0 1 1,e,2330,2783,test
1,51759b5579bc_image,none 1 0 0 1 1,e,3093,2850,test


In [68]:
shutil.copytree('/kaggle/input/yolov5-official-v31-dataset/yolov5', '/kaggle/working/yolov5')
os.chdir('/kaggle/working/yolov5') # install dependencies

## yolov5 fold 0

In [69]:
dim = 640 #1024, 256, 'original'
test_dir = f'/kaggle/tmp/{split}/image'
weights_dir = '/kaggle/input/siimcovid19-detection-models/yolov5x_fold0_nb_v14_best.pt'

import torch
#from IPython.display import Image, clear_output  # to display images

#clear_output()
#print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))


!python detect.py --weights $weights_dir\
--img 640\
--conf 0.001\
--iou 0.5\
--source $test_dir\
--save-txt --save-conf --augment


def yolo2voc(image_height, image_width, bboxes):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    voc  => [x1, y1, x2, y1]

    """ 
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int

    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height

    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]

    return bboxes

image_ids = [] # (num_images, )
PredictionStrings = [] # (num_images, boxes_lst)

for file_path in tqdm(glob('runs/detect/exp/labels/*.txt')):
    image_id = file_path.split('/')[-1].split('.')[0]
    w, h = test_df.loc[test_df.id==image_id,['dim1', 'dim0']].values[0]
    f = open(file_path, 'r')
    data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
    data = data[:, [0, 5, 1, 2, 3, 4]]
    bboxes = list(np.round(np.concatenate((data[:, :2], np.round(yolo2voc(h, w, data[:, 2:]))), axis =1).reshape(-1), 12).astype(str))
    new_bboxes = []
    for idx in range(len(bboxes)):
        if idx%6 == 0:
            new_bboxes.append('opacity')
        else :
            new_bboxes.append(bboxes[idx])
    image_ids.append(image_id)
    PredictionStrings.append(' '.join(new_bboxes))


pred_yolov5_fold0_df = pd.DataFrame({'id':image_ids,
                        'PredictionString':PredictionStrings})
pred_yolov5_fold0_df.head()

Namespace(agnostic_nms=False, augment=True, classes=None, conf_thres=0.001, device='', exist_ok=False, img_size=640, iou_thres=0.5, name='exp', project='runs/detect', save_conf=True, save_txt=True, source='/kaggle/tmp/test/image', update=False, view_img=False, weights=['/kaggle/input/siimcovid19-detection-models/yolov5x_fold0_nb_v14_best.pt'])
Fusing layers... 
image 1/2 /kaggle/tmp/test/image/51759b5579bc_image.png: 640x640 50 0. opacitys, Done. (0.164s)
image 2/2 /kaggle/tmp/test/image/65761e66de9f_image.png: 640x640 41 0. opacitys, Done. (0.156s)
Results saved to runs/detect/exp
2 labels saved to runs/detect/exp/labels
Done. (0.554s)


  0%|          | 0/2 [00:00<?, ?it/s]

,id,PredictionString
0,51759b5579bc_image,opacity 0.001049999963 1585.0 764.0 2195.0 176...
1,65761e66de9f_image,opacity 0.001011849963 2061.0 1154.0 2366.0 14...


## yolov5 fold 1

In [70]:
dim = 640 #1024, 256, 'original'
test_dir = f'/kaggle/tmp/{split}/image'
weights_dir = '/kaggle/input/siimcovid19-detection-models/yolov5x_fold1_nb_v19_best.pt'

import torch
#from IPython.display import Image, clear_output  # to display images

#clear_output()
#print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))


!python detect.py --weights $weights_dir\
--img 640\
--conf 0.001\
--iou 0.5\
--source $test_dir\
--save-txt --save-conf --augment


def yolo2voc(image_height, image_width, bboxes):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    voc  => [x1, y1, x2, y1]

    """ 
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int

    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height

    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]

    return bboxes

image_ids = [] # (num_images, )
PredictionStrings = [] # (num_images, boxes_lst)

for file_path in tqdm(glob('runs/detect/exp2/labels/*.txt')):
    image_id = file_path.split('/')[-1].split('.')[0]
    w, h = test_df.loc[test_df.id==image_id,['dim1', 'dim0']].values[0]
    f = open(file_path, 'r')
    data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
    data = data[:, [0, 5, 1, 2, 3, 4]]
    bboxes = list(np.round(np.concatenate((data[:, :2], np.round(yolo2voc(h, w, data[:, 2:]))), axis =1).reshape(-1), 12).astype(str))
    new_bboxes = []
    for idx in range(len(bboxes)):
        if idx%6 == 0:
            new_bboxes.append('opacity')
        else :
            new_bboxes.append(bboxes[idx])
    image_ids.append(image_id)
    PredictionStrings.append(' '.join(new_bboxes))


pred_yolov5_fold1_df = pd.DataFrame({'id':image_ids,
                        'PredictionString':PredictionStrings})
pred_yolov5_fold1_df.head()

Namespace(agnostic_nms=False, augment=True, classes=None, conf_thres=0.001, device='', exist_ok=False, img_size=640, iou_thres=0.5, name='exp', project='runs/detect', save_conf=True, save_txt=True, source='/kaggle/tmp/test/image', update=False, view_img=False, weights=['/kaggle/input/siimcovid19-detection-models/yolov5x_fold1_nb_v19_best.pt'])
Fusing layers... 
image 1/2 /kaggle/tmp/test/image/51759b5579bc_image.png: 640x640 40 0. opacitys, Done. (0.196s)
image 2/2 /kaggle/tmp/test/image/65761e66de9f_image.png: 640x640 33 0. opacitys, Done. (0.166s)
Results saved to runs/detect/exp2
2 labels saved to runs/detect/exp2/labels
Done. (0.595s)


  0%|          | 0/2 [00:00<?, ?it/s]

,id,PredictionString
0,51759b5579bc_image,opacity 0.001026149956 414.0 1875.0 815.0 2368...
1,65761e66de9f_image,opacity 0.001026149956 848.0 932.0 1148.0 1300...


## yolov5 fold 2

In [71]:
dim = 640 #1024, 256, 'original'
test_dir = f'/kaggle/tmp/{split}/image'
weights_dir = '/kaggle/input/siimcovid19-detection-models/yolov5x_fold2_nb_v20_best.pt'

import torch
#from IPython.display import Image, clear_output  # to display images

#clear_output()
#print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))


!python detect.py --weights $weights_dir\
--img 640\
--conf 0.001\
--iou 0.5\
--source $test_dir\
--save-txt --save-conf --augment


def yolo2voc(image_height, image_width, bboxes):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    voc  => [x1, y1, x2, y1]

    """ 
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int

    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height

    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]

    return bboxes

image_ids = [] # (num_images, )
PredictionStrings = [] # (num_images, boxes_lst)

for file_path in tqdm(glob('runs/detect/exp3/labels/*.txt')):
    image_id = file_path.split('/')[-1].split('.')[0]
    w, h = test_df.loc[test_df.id==image_id,['dim1', 'dim0']].values[0]
    f = open(file_path, 'r')
    data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
    data = data[:, [0, 5, 1, 2, 3, 4]]
    bboxes = list(np.round(np.concatenate((data[:, :2], np.round(yolo2voc(h, w, data[:, 2:]))), axis =1).reshape(-1), 12).astype(str))
    new_bboxes = []
    for idx in range(len(bboxes)):
        if idx%6 == 0:
            new_bboxes.append('opacity')
        else :
            new_bboxes.append(bboxes[idx])
    image_ids.append(image_id)
    PredictionStrings.append(' '.join(new_bboxes))


pred_yolov5_fold2_df = pd.DataFrame({'id':image_ids,
                        'PredictionString':PredictionStrings})
pred_yolov5_fold2_df.head()

Namespace(agnostic_nms=False, augment=True, classes=None, conf_thres=0.001, device='', exist_ok=False, img_size=640, iou_thres=0.5, name='exp', project='runs/detect', save_conf=True, save_txt=True, source='/kaggle/tmp/test/image', update=False, view_img=False, weights=['/kaggle/input/siimcovid19-detection-models/yolov5x_fold2_nb_v20_best.pt'])
Fusing layers... 
image 1/2 /kaggle/tmp/test/image/51759b5579bc_image.png: 640x640 42 0. opacitys, Done. (0.178s)
image 2/2 /kaggle/tmp/test/image/65761e66de9f_image.png: 640x640 36 0. opacitys, Done. (0.155s)
Results saved to runs/detect/exp3
2 labels saved to runs/detect/exp3/labels
Done. (0.537s)


  0%|          | 0/2 [00:00<?, ?it/s]

,id,PredictionString
0,51759b5579bc_image,opacity 0.001051899977 1986.0 1073.0 2498.0 18...
1,65761e66de9f_image,opacity 0.001035689958 1974.0 815.0 2366.0 124...


## yolov5 fold 3

In [72]:
# dim = 640 #1024, 256, 'original'
# test_dir = f'/kaggle/tmp/{split}/image'
# weights_dir = '/kaggle/input/siimcovid19-detection-models/yolov5x_fold3_nb_v21_best.pt'

# import torch
# #from IPython.display import Image, clear_output  # to display images

# #clear_output()
# #print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))


# !python detect.py --weights $weights_dir\
# --img 640\
# --conf 0.001\
# --iou 0.5\
# --source $test_dir\
# --save-txt --save-conf --augment


# def yolo2voc(image_height, image_width, bboxes):
#     """
#     yolo => [xmid, ymid, w, h] (normalized)
#     voc  => [x1, y1, x2, y1]

#     """ 
#     bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int

#     bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
#     bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height

#     bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
#     bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]

#     return bboxes

# image_ids = [] # (num_images, )
# PredictionStrings = [] # (num_images, boxes_lst)

# for file_path in tqdm(glob('runs/detect/exp4/labels/*.txt')):
#     image_id = file_path.split('/')[-1].split('.')[0]
#     w, h = test_df.loc[test_df.id==image_id,['dim1', 'dim0']].values[0]
#     f = open(file_path, 'r')
#     data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
#     data = data[:, [0, 5, 1, 2, 3, 4]]
#     bboxes = list(np.round(np.concatenate((data[:, :2], np.round(yolo2voc(h, w, data[:, 2:]))), axis =1).reshape(-1), 12).astype(str))
#     new_bboxes = []
#     for idx in range(len(bboxes)):
#         if idx%6 == 0:
#             new_bboxes.append('opacity')
#         else :
#             new_bboxes.append(bboxes[idx])
#     image_ids.append(image_id)
#     PredictionStrings.append(' '.join(new_bboxes))


# pred_yolov5_fold3_df = pd.DataFrame({'id':image_ids,
#                         'PredictionString':PredictionStrings})
# pred_yolov5_fold3_df.head()

## yolov5 fold 4

In [73]:
# dim = 640 #1024, 256, 'original'
# test_dir = f'/kaggle/tmp/{split}/image'
# weights_dir = '/kaggle/input/siimcovid19-detection-models/yolov5x_fold4_nb_v22_best.pt'

# import torch
# #from IPython.display import Image, clear_output  # to display images

# #clear_output()
# #print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))


# !python detect.py --weights $weights_dir\
# --img 640\
# --conf 0.001\
# --iou 0.5\
# --source $test_dir\
# --save-txt --save-conf --augment


# def yolo2voc(image_height, image_width, bboxes):
#     """
#     yolo => [xmid, ymid, w, h] (normalized)
#     voc  => [x1, y1, x2, y1]

#     """ 
#     bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int

#     bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
#     bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height

#     bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
#     bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]

#     return bboxes

# image_ids = [] # (num_images, )
# PredictionStrings = [] # (num_images, boxes_lst)

# for file_path in tqdm(glob('runs/detect/exp5/labels/*.txt')):
#     image_id = file_path.split('/')[-1].split('.')[0]
#     w, h = test_df.loc[test_df.id==image_id,['dim1', 'dim0']].values[0]
#     f = open(file_path, 'r')
#     data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
#     data = data[:, [0, 5, 1, 2, 3, 4]]
#     bboxes = list(np.round(np.concatenate((data[:, :2], np.round(yolo2voc(h, w, data[:, 2:]))), axis =1).reshape(-1), 12).astype(str))
#     new_bboxes = []
#     for idx in range(len(bboxes)):
#         if idx%6 == 0:
#             new_bboxes.append('opacity')
#         else :
#             new_bboxes.append(bboxes[idx])
#     image_ids.append(image_id)
#     PredictionStrings.append(' '.join(new_bboxes))


# pred_yolov5_fold4_df = pd.DataFrame({'id':image_ids,
#                         'PredictionString':PredictionStrings})
# pred_yolov5_fold4_df.head()

# MMDetection

In [74]:
!pip install /kaggle/input/mmdet2100/mmdetection-2.10.0/addict-2.4.0-py3-none-any.whl
!pip install /kaggle/input/mmdet2100/mmdetection-2.10.0/yapf-0.31.0-py2.py3-none-any.whl
!pip install /kaggle/input/pycocotools202/pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl
!pip install /kaggle/input/mmcvfull134/mmcv_full-1.3.4-cp37-cp37m-manylinux1_x86_64.whl
!pip install /kaggle/input/mmdetection2120/mmdetection-2.12.0 -f ./ --no-index

Processing /kaggle/input/mmdet2100/mmdetection-2.10.0/addict-2.4.0-py3-none-any.whl
Processing /kaggle/input/mmdet2100/mmdetection-2.10.0/yapf-0.31.0-py2.py3-none-any.whl
Processing /kaggle/input/pycocotools202/pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl
  Attempting uninstall: pycocotools
    Found existing installation: pycocotools 2.0
    Uninstalling pycocotools-2.0:
      Successfully uninstalled pycocotools-2.0
Processing /kaggle/input/mmcvfull134/mmcv_full-1.3.4-cp37-cp37m-manylinux1_x86_64.whl
Looking in links: ./
Processing /kaggle/input/mmdetection2120/mmdetection-2.12.0
  Created wheel for mmdet: filename=mmdet-2.12.0-py3-none-any.whl size=592092 sha256=f378fff1dcc1706a4d62eba3848f42310eaa3532d8d5e70b918bca0e834764c0
  Stored in directory: /root/.cache/pip/wheels/05/b5/eb/d376ff42b05cded8c6f10ef2f138c8e0a68f40fcdbd33e8c15
Successfully built mmdet


In [75]:
import os
import shutil
import yaml
import time
import json
import cv2
import random
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold, StratifiedKFold
from tqdm.notebook import tqdm
import seaborn as sns
import torch
from IPython.display import Image, clear_output
from collections import Counter
from ensemble_boxes import *
import copy
import os.path as osp
import mmcv
import mmdet
import numpy as np
import albumentations as A
from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset
from mmcv import Config
from mmdet.apis import set_random_seed
from mmdet.apis import inference_detector, init_detector, show_result_pyplot
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
print(mmdet.__version__)
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

2.12.0


In [76]:
checkpoint = '/kaggle/input/siimcovid19-detection-models/cascade_rcnn_x101_32x4d_fpn_1x_fold0_epoch10_public_nb_v8.pth'
cfg = '/kaggle/input/siimcovid19-detection-models/cascade_rcnn_x101_32x4d_fpn_1x_fold0_epoch10_public_nb_v8_config.py'

cfg = Config.fromfile(cfg)

cfg.classes = ("Covid_Abnormality")
cfg.data.test.img_prefix = ''
cfg.data.test.classes = cfg.classes

# cfg.model.roi_head.bbox_head.num_classes = 1
# cfg.model.bbox_head.num_classes = 1
for head in cfg.model.roi_head.bbox_head:
    head.num_classes = 1

# Set seed thus the results are more reproducible
cfg.seed = 211
set_random_seed(211, deterministic=False)
cfg.gpu_ids = [0]

cfg.data.test.pipeline=[
            dict(type='LoadImageFromFile'),
            dict(
                type='MultiScaleFlipAug',
                img_scale=(1333, 800),
                flip=False,
                transforms=[
                    dict(type='Resize', keep_ratio=True),
                    dict(type='RandomFlip', direction='horizontal'),
                    dict(
                        type='Normalize',
                        mean=[123.675, 116.28, 103.53],
                        std=[58.395, 57.12, 57.375],
                        to_rgb=True),
                    dict(type='Pad', size_divisor=32),
                    dict(type='DefaultFormatBundle'),
                    dict(type='Collect', keys=['img'])
                ])
        ]

cfg.test_pipeline = [
            dict(type='LoadImageFromFile'),
            dict(
                type='MultiScaleFlipAug',
                img_scale=(1333, 800),
                flip=False,
                transforms=[
                    dict(type='Resize', keep_ratio=True),
                    dict(type='RandomFlip', direction='horizontal'),
                    dict(
                        type='Normalize',
                        mean=[123.675, 116.28, 103.53],
                        std=[58.395, 57.12, 57.375],
                        to_rgb=True),
                    dict(type='Pad', size_divisor=32),
                    dict(type='DefaultFormatBundle'),
                    dict(type='Collect', keys=['img'])
                ])
        ]

# cfg.data.samples_per_gpu = 4
# cfg.data.workers_per_gpu = 4
# cfg.model.test_cfg.nms.iou_threshold = 0.3
cfg.model.test_cfg.rcnn.score_thr = 0.001

model_test = init_detector(cfg, checkpoint, device='cuda:0')
print(model_test)

Use load_from_local loader
CascadeRCNN(
  (backbone): ResNeXt(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): ResLayer(
      (0): Bottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=Tr

/opt/conda/lib/python3.7/site-packages/mmdet/apis/inference.py:47: UserWarning: Class names are not saved in the checkpoint's meta data, use COCO classes by default.
  warnings.warn('Class names are not saved in the checkpoint\'s '


In [77]:
def nms_one_img(preds, img_path, img_width, img_height):
    # print('img width:', img_width)
    # print('img height:', img_height)
    boxes_list = []
    scores_list = []
    labels_list = []
    weights = [1]
    # print(preds)
    for i, pred in enumerate(preds):
        if len(pred):
            for p in pred:
                box = [0, 0, 0, 0]
                box[0] = min(1.0, p[0] / img_width)
                box[1] = min(1.0, p[1] / img_height)
                box[2] = min(1.0, p[2] / img_width)
                box[3] = min(1.0, p[3] / img_height)
                # print(box)
                for b in box:
                    if b > 1:
                        print(img_path)
                boxes_list.append(box)
                score = p[4].astype(float)
                scores_list.append(score)
                labels_list.append(i)
    # print('Before:')
    # print(boxes_list)
    boxes_list, scores_list, labels_list = nms([boxes_list], [scores_list], [labels_list], weights=weights, iou_thr=0.6)
    # print('After:')
    # print(boxes_list)
    return boxes_list, scores_list, labels_list

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [78]:
def infer_img(model, img_path):
    img = mmcv.imread(img_path)
    result = inference_detector(model, img_path)
    # print(img.shape)
    # print(img.shape[0])
    # print(img.shape[1])
    boxes_list, scores_list, labels_list = nms_one_img(result, img_path, img.shape[1], img.shape[0])
    return boxes_list, scores_list, labels_list

In [79]:
def round_float(str_float, dec=3):
    return str(round(float(str_float), 3))

In [80]:
def convert_to_df_row_list(img_id, boxes_list, scores_list, labels_list):
    res = []
    res.append(img_id)
    prediction_string = ''
    for i, label in enumerate(labels_list):
        if int(label) == 1:
            prediction_string = 'none 1 0 0 1 1'
            continue
        else:
            prediction_string += 'opacity'
        prediction_string += ' '
        prediction_string += str(round(float(scores_list[i]), 3))
        prediction_string += ' '
        prediction_string += round_float(boxes_list[i][0], 3) + ' ' + round_float(boxes_list[i][1], 3) + ' ' + round_float(boxes_list[i][2], 3) + ' ' + round_float(boxes_list[i][3], 3) + ' '
    res.append(prediction_string.rstrip())
    return res

In [81]:
vfnet_pred_lst = []
test_dir = f'/kaggle/tmp/{split}/image'
count = 0

for img_name in os.listdir(test_dir):
    img_path = test_dir + '/' + img_name
    # boxes_list, scores_list, labels_list = infer_img(model, '../vinbigdata-cocodataset/ori_vinbigdata_3xdownsampled/test/test/b461cd28bc17c294dd986d0d91577ac3.jpg')
    boxes_list, scores_list, labels_list = infer_img(model_test, img_path)
    # print(boxes_list)
    # print(scores_list)
    # print(labels_list)
    image_id = img_name.split('.png')[0]
    ori_img_width = test_df[test_df.id == image_id]['dim1'].iloc[0]
    ori_img_height = test_df[test_df.id == image_id]['dim0'].iloc[0]
    boxes_list[:, 0] = boxes_list[:, 0] * ori_img_width
    boxes_list[:, 2] = boxes_list[:, 2] * ori_img_width
    boxes_list[:, 1] = boxes_list[:, 1] * ori_img_height
    boxes_list[:, 3] = boxes_list[:, 3] * ori_img_height
    df_row_lst = convert_to_df_row_list(image_id, boxes_list, scores_list, labels_list)
    vfnet_pred_lst.append(df_row_lst)

In [82]:
vfnet_pred_df = pd.DataFrame(vfnet_pred_lst, columns=['id', 'PredictionString'])
vfnet_pred_df.head()

,id,PredictionString
0,65761e66de9f_image,opacity 0.805 1810.332 806.626 2422.337 1523.4...
1,51759b5579bc_image,opacity 0.747 158.294 1842.22 1015.081 2506.00...


# Detection Emsemble

In [83]:
# %%
import numpy as np
import pandas as pd
from ensemble_boxes import nms, soft_nms, non_maximum_weighted, weighted_boxes_fusion

# %%
''' Merge output of two models after 2cls filter
- Detectron2: https://www.kaggle.com/corochann/vinbigdata-detectron2-prediction
- Yolov5: https://www.kaggle.com/awsaf49/vinbigdata-2-class-filter
Reference:
- https://github.com/ZFTurbo/Weighted-Boxes-Fusion
'''

# %%
weights = [54, 54, 54, 58, 58, 54]
iou_thr = 0.6
skip_box_thr = 0.01 # 0.0001 for non soft-nms
sigma = 0.1

# %%
test_meta = test_df
merged_df = pd.DataFrame(columns=['id', 'PredictionString'])

# %%
'''Weighted Boxes Fusion'''
image_id_lst = pred_yolov5_fold0_df['id'].unique()

# %%
# Helper functions
def extract_data(data, img_height, img_width):
    boxes_lst = []
    scores_lst = []
    labels_lst = []
    data_lst = data.split(' ')
    for i in range(0, len(data_lst), 6):
        labels_lst.append(0)
        scores_lst.append(float(data_lst[i + 1]))
        x_min = float(data_lst[i + 2]) / img_width
        y_min = float(data_lst[i + 3]) / img_height
        x_max = float(data_lst[i + 4]) / img_width
        y_max = float(data_lst[i + 5]) / img_height
        boxes_lst.append([x_min, y_min, x_max, y_max])
    return boxes_lst, scores_lst, labels_lst

def convert_data_to_row(boxes, scores, labels):
    data_lst = []
    for i in range(len(boxes)):
        data_lst.append('opacity')
        data_lst.append(str(scores[i]))
        data_lst.append(str(boxes[i][0]))
        data_lst.append(str(boxes[i][1]))
        data_lst.append(str(boxes[i][2]))
        data_lst.append(str(boxes[i][3]))
    data = ' '.join(data_lst)
    return data

def get_height_width(image_id):
    # dim0: heigth, dim1: width
    height = test_meta[test_meta['id'] == image_id]['dim0'].values[0]
    width = test_meta[test_meta['id'] == image_id]['dim1'].values[0]
    return height, width

def scale_data(boxes, img_height, img_width):
    res = []
    for box in boxes:
        temp = []
        temp.append(box[0] * img_width)
        temp.append(box[1] * img_height)
        temp.append(box[2] * img_width)
        temp.append(box[3] * img_height)
        res.append(temp)
    return res

# %%
def wbf(image_id):
    img_height, img_width = get_height_width(image_id)
    boxes_lst, scores_lst, labels_lst = [], [], []

    pred_yolov5_fold0_data = pred_yolov5_fold0_df[pred_yolov5_fold0_df['id'] == image_id]['PredictionString'].values[0]
    model_boxes_lst, model_scores_lst, model_labels_lst = extract_data(pred_yolov5_fold0_data, img_height, img_width)
    boxes_lst.append(model_boxes_lst)
    scores_lst.append(model_scores_lst)
    labels_lst.append(model_labels_lst)

    pred_yolov5_fold1_data = pred_yolov5_fold1_df[pred_yolov5_fold1_df['id'] == image_id]['PredictionString'].values[0]
    model_boxes_lst, model_scores_lst, model_labels_lst = extract_data(pred_yolov5_fold1_data, img_height, img_width)
    boxes_lst.append(model_boxes_lst)
    scores_lst.append(model_scores_lst)
    labels_lst.append(model_labels_lst)

    pred_yolov5_fold2_data = pred_yolov5_fold2_df[pred_yolov5_fold2_df['id'] == image_id]['PredictionString'].values[0]
    model_boxes_lst, model_scores_lst, model_labels_lst = extract_data(pred_yolov5_fold2_data, img_height, img_width)
    boxes_lst.append(model_boxes_lst)
    scores_lst.append(model_scores_lst)
    labels_lst.append(model_labels_lst)

#     pred_yolov5_fold3_data = pred_yolov5_fold3_df[pred_yolov5_fold3_df['id'] == image_id]['PredictionString'].values[0]
#     model_boxes_lst, model_scores_lst, model_labels_lst = extract_data(pred_yolov5_fold3_data, img_height, img_width)
#     boxes_lst.append(model_boxes_lst)
#     scores_lst.append(model_scores_lst)
#     labels_lst.append(model_labels_lst)

#     pred_yolov5_fold4_data = pred_yolov5_fold4_df[pred_yolov5_fold4_df['id'] == image_id]['PredictionString'].values[0]
#     model_boxes_lst, model_scores_lst, model_labels_lst = extract_data(pred_yolov5_fold4_data, img_height, img_width)
#     boxes_lst.append(model_boxes_lst)
#     scores_lst.append(model_scores_lst)
#     labels_lst.append(model_labels_lst)
    
#     effdet_fold0_data = effdet_fold0[effdet_fold0['id'] == image_id]['PredictionString'].values[0]
#     model_boxes_lst, model_scores_lst, model_labels_lst = extract_data(effdet_fold0_data, img_height, img_width)
#     boxes_lst.append(model_boxes_lst)
#     scores_lst.append(model_scores_lst)
#     labels_lst.append(model_labels_lst)
    
#     effdet_fold1_data = effdet_fold1[effdet_fold1['id'] == image_id]['PredictionString'].values[0]
#     model_boxes_lst, model_scores_lst, model_labels_lst = extract_data(effdet_fold1_data, img_height, img_width)
#     boxes_lst.append(model_boxes_lst)
#     scores_lst.append(model_scores_lst)
#     labels_lst.append(model_labels_lst)
    
#     effdet_fold2_data = effdet_fold2[effdet_fold2['id'] == image_id]['PredictionString'].values[0]
#     model_boxes_lst, model_scores_lst, model_labels_lst = extract_data(effdet_fold2_data, img_height, img_width)
#     boxes_lst.append(model_boxes_lst)
#     scores_lst.append(model_scores_lst)
#     labels_lst.append(model_labels_lst)
    
    effdet_fold3_data = effdet_fold3[effdet_fold3['id'] == image_id]['PredictionString'].values[0]
    model_boxes_lst, model_scores_lst, model_labels_lst = extract_data(effdet_fold3_data, img_height, img_width)
    boxes_lst.append(model_boxes_lst)
    scores_lst.append(model_scores_lst)
    labels_lst.append(model_labels_lst)
    
    effdet_fold4_data = effdet_fold4[effdet_fold4['id'] == image_id]['PredictionString'].values[0]
    model_boxes_lst, model_scores_lst, model_labels_lst = extract_data(effdet_fold4_data, img_height, img_width)
    boxes_lst.append(model_boxes_lst)
    scores_lst.append(model_scores_lst)
    labels_lst.append(model_labels_lst)
    
    vfnet_pred_data = vfnet_pred_df[vfnet_pred_df['id'] == image_id]['PredictionString'].values[0]
    model_boxes_lst, model_scores_lst, model_labels_lst = extract_data(vfnet_pred_data, img_height, img_width)
    boxes_lst.append(model_boxes_lst)
    scores_lst.append(model_scores_lst)
    labels_lst.append(model_labels_lst)

    boxes, scores, labels = weighted_boxes_fusion(boxes_lst, scores_lst, labels_lst, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    # boxes, scores, labels = non_maximum_weighted(boxes_lst, scores_lst, labels_lst, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    # boxes, scores, labels = nms(boxes_lst, scores_lst, labels_lst, weights=weights, iou_thr=iou_thr)
    # boxes, scores, labels = soft_nms(boxes_lst, scores_lst, labels_lst, weights=weights, iou_thr=iou_thr, sigma=sigma, thresh=skip_box_thr)
    boxes = scale_data(boxes, img_height, img_width)
    # return boxes, scores, labels
    merged_data = convert_data_to_row(boxes, scores, labels)
    merged_data = pd.DataFrame([[image_id, merged_data]], columns=['id', 'PredictionString'])
    return merged_data

# %%
# Test
# test_boxes, test_scores, test_labels = wbf(image_id_lst[0])
test_wbf = wbf(image_id_lst[0])

# %%
for image_id in image_id_lst:
    merged_data = wbf(image_id)
    merged_df = merged_df.append(merged_data, ignore_index=True)

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [84]:
merged_df

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,id,PredictionString
0,51759b5579bc_image,opacity 0.3920193612575531 159.31273866444826 ...
1,65761e66de9f_image,opacity 0.3600524067878723 1834.7672883868217 ...


In [85]:
test_df

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,id,PredictionString,id_last_str,dim0,dim1,split
0,65761e66de9f_image,none 1 0 0 1 1,e,2330,2783,test
1,51759b5579bc_image,none 1 0 0 1 1,e,3093,2850,test


In [86]:
test_df = test_df.drop(['PredictionString'], axis=1)
sub_df = pd.merge(test_df, merged_df, on = 'id', how = 'left').fillna("none 1 0 0 1 1")
sub_df

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,id,id_last_str,dim0,dim1,split,PredictionString
0,65761e66de9f_image,e,2330,2783,test,opacity 0.3600524067878723 1834.7672883868217 ...
1,51759b5579bc_image,e,3093,2850,test,opacity 0.3920193612575531 159.31273866444826 ...


In [87]:
sub_df = sub_df[['id', 'PredictionString']]
sub_df['none'] = df_2class['none']
sub_df

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id,PredictionString,none
0,65761e66de9f_image,opacity 0.3600524067878723 1834.7672883868217 ...,0.129468
1,51759b5579bc_image,opacity 0.3920193612575531 159.31273866444826 ...,0.753524


In [88]:
for i in range(sub_df.shape[0]):
    if sub_df.loc[i,'PredictionString'] == "none 1 0 0 1 1":
        continue
    sub_df_split = sub_df.loc[i,'PredictionString'].split()
    none_score = float(sub_df.loc[i, 'none'])
    sub_df_list = []
    for j in range(int(len(sub_df_split) / 6)):
        sub_df_list.append('opacity')
        opacity_score = float(sub_df_split[6 * j + 1])
        calibrated_opacity_score = opacity_score * ((1 - none_score) ** 0.5) 
        sub_df_list.append(str(calibrated_opacity_score))
        sub_df_list.append(sub_df_split[6 * j + 2])
        sub_df_list.append(sub_df_split[6 * j + 3])
        sub_df_list.append(sub_df_split[6 * j + 4])
        sub_df_list.append(sub_df_split[6 * j + 5])
    sub_df.loc[i,'PredictionString'] = ' '.join(sub_df_list)
for i in range(sub_df.shape[0]):
    if sub_df.loc[i,'PredictionString'] != 'none 1 0 0 1 1':
        sub_df.loc[i,'PredictionString'] = sub_df.loc[i,'PredictionString'] + ' none ' + str(sub_df.loc[i,'none']) + ' 0 0 1 1'
sub_df = sub_df[['id', 'PredictionString']]   
df_study = df_study[:study_len]
df_study = df_study.append(sub_df).reset_index(drop=True)
df_study.to_csv('/kaggle/working/submission.csv',index = False)  
shutil.rmtree('/kaggle/working/yolov5')

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [89]:
df_study

,id,PredictionString
0,00086460a852_study,negative 0.112 0 0 1 1 indeterminate 0.222 0 0...
1,000c9c05fd14_study,negative 0.042 0 0 1 1 indeterminate 0.053 0 0...
2,65761e66de9f_image,opacity 0.33593724537384795 1834.7672883868217...
3,51759b5579bc_image,opacity 0.19462346405137868 159.31273866444826...
